# Question 1. Data Classification with HMM


In [1]:
import numpy as np
import pandas as pd
from pomegranate import *

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import torch.utils.data as data_utils

Read the entire data set
The dataset is obtained from: [Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).
> It originally consists of 6 different activities: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
> For this assignment we are considering only three activities: WALKING, WALKING_UPSTAIRS, and SITTING.


In [2]:
#****1b******: Code is added to read the training and test data
train = pd.read_excel('Train_q2.xlsx')
test = pd.read_excel('Test_q2.xlsx')

#1b. Create dataframes for each class label

> Note the labels are: 1, 2 and 4.

In [3]:
#****1b******Add code here: Create frames for the 3 labels
import collections
def preProcess(matrix):
    # 2947*562
    m=len(matrix)
    n=len(matrix[0])
    one=[]
    two=[]
    three=[]
    
    for i in range(m):
        label = int(matrix[i][-1])
        if label==1:      #1=walking; 2=walking_downstairs; 4=sitting
            one.append(matrix[i][:-1])
        elif label==2:
            two.append(matrix[i][:-1])
        elif label==4:
            three.append(matrix[i][:-1])
#                 Y.append(abs(y-4))  #we want to store 1 for happy, 0 for sad
#                 X.append([int(p) for p in row[1].split()])
    return one,two,three
#     since all the values are between 0 and 1, normalization is needless
#     return np.array(X) / 255.0, np.array(Y)

np_train=np.array(train)
py_train=np_train.tolist()
one,two,three=preProcess(py_train)

np_one=np.array(one)
train_one=pd.DataFrame(np_one)
print(train_one.shape)

np_two=np.array(two)
train_two=pd.DataFrame(np_two)
print(train_two.shape)

np_three=np.array(three)
train_three=pd.DataFrame(np_three)
print(train_three.shape)


(1226, 561)
(1073, 561)
(1286, 561)


Drop the 'label' column

In [4]:
#****1b******Add code here: Drop the labels for your 3 dataframes
# X = train.loc[:, train.columns != 'label']
# X_test = test.loc[:, test.columns != 'label']
# y = train[['label']]
# y_test = test[['label']]

Convert the dataframes to numpy arrays

In [5]:
#****1b******Add code here: Convert the frames to numpy arrays
final_one=np.array(train_one)
final_two=np.array(train_two)
final_three=np.array(train_three)

#1c. Create HMM models for each class
We are using the baum-welch algorithm to find the parameters of the model.

In [6]:
#****1c******Add code here: learn the parameters of 3 different HMMs
model1=HiddenMarkovModel.from_samples(NormalDistribution, n_components=3, X=final_one)
model2=HiddenMarkovModel.from_samples(NormalDistribution, n_components=3, X=final_two)
model3=HiddenMarkovModel.from_samples(NormalDistribution, n_components=3, X=final_three)

#1d. Check Performance of the Models

For each test sample we will get the log_probabilities of the sample belonging to each of the three classes.

We will assign the sample to the model which gives the highest log_probability value

In [9]:
def get_model_accuracy(df, class_no):
#****1d******Add code here: 
    '''
    Accepts the numpy array belonging to a class and the class label which we want to
    use to test our accurracy.
    '''
    cnt=[[0]*3 for _ in range(3)]
    for i in range(len(df)):
        one=model1.log_probability(df[i])
        two=model2.log_probability(df[i])
        three=model3.log_probability(df[i])
        target=-1
        if int(class_no[i])==1:
            target=0
        elif int(class_no[i])==2:
            target=1
        else:
            target=2
            
        if one>=max(two,three):
            judge=0
            cnt[target][judge]+=1
        elif two>=max(one,three):
            judge=1
            cnt[target][judge]+=1
        elif three>=max(one,two):
            judge=2
            cnt[target][judge]+=1
    return cnt

np_test=np.array(test)
py_test=np_test.tolist()

df=[]
class_no=[]
for i in range(len(py_test)):
    if py_test[i][-1]==1 or py_test[i][-1]==2 or py_test[i][-1]==4:
        df.append(py_test[i][:-1])
        class_no.append(py_test[i][-1])
    
df=np.array(df)
class_no=np.array(class_no)

cnt=get_model_accuracy(df,class_no)

In [10]:
#****1e******Add code here: Generate the confusion matrix
print(cnt)

[[299, 197, 0], [177, 294, 0], [0, 43, 448]]
